# NBA Models

## Data Export
```
# player
dumpdata.sc --seasons 20152016 20162017 20172018 20182019 20192020 20202021 20212022 \
    --stats "*" --progress \
    nba_hist_20082009-20212022.scored.db --no_teams --target_calc_stats "*" \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home -f nba_player.csv

# team
dumpdata.sc --seasons 20152016 20162017 20172018 20182019 20192020 20202021 20212022 \
    --stats "*" --progress \
    nba_hist_20082009-20212022.scored.db --no_players --target_stats pts win \
    --hist_recent_games 5 --hist_recent_mode ma --current_extra is_home -f nba_team.csv
```

In [ ]:
RANDOM_SEED = 1
TRAINING_TIME = 600
VALIDATION_SEASON = 20212022
RECENT_GAMES = 5
TRAINING_SEASONS = [20152016, 20162017, 20172018, 20182019, 20192020, 20202021]

In [ ]:
from datetime import datetime

from fantasy_py import PlayerOrTeam
from train_test import load_data, train_test, create_fantasy_model

TARGET = ("stat", "pts")
MODEL_NAME = "NBA-team-pts"
DATA_FILENAME = "/fantasy/nba_team.csv"
P_OR_T = PlayerOrTeam.TEAM
INCLUDE_POS = None

# TARGET = ("stat", "win")
# MODEL_NAME = "NBA-team-win"
# DATA_FILENAME = "/fantasy/nba_team.csv"
# P_OR_T = "T"
# INCLUDE_POS = None

# TARGET = ("calc", "dk_score")
# MODEL_NAME = "NBA-DK"
# DATA_FILENAME = "/fantasy/nba_player.csv"
# P_OR_T = PlayerOrTeam.PLAYER
# INCLUDE_POS = True

df, tt_data = load_data(
    DATA_FILENAME,
    TARGET,
    VALIDATION_SEASON,
    seed=RANDOM_SEED,
    include_position=INCLUDE_POS,
)

for automl_type in ["tpot"]:  # , "autosk"]:
    dt_trained = datetime.now()

    model_filepath, performance = train_test(
        automl_type, MODEL_NAME, TARGET, tt_data, RANDOM_SEED, TRAINING_TIME, dt_trained
    )
    performance["season"] = VALIDATION_SEASON
    model = create_fantasy_model(
        MODEL_NAME,
        model_filepath,
        dt_trained,
        tt_data[0],
        TARGET,
        TRAINING_TIME,
        P_OR_T,
        RECENT_GAMES,
        automl_type,
        performance,
        TRAINING_SEASONS,
        seed=RANDOM_SEED,
    )
    model_filepath = model.dump(".".join([MODEL_NAME, TARGET[1], automl_type, "model"]))
    print(f"Model file saved to '{model_filepath}'")
